libraries

In [26]:
from re import sub
from skimage.io import  imread, imshow
from skimage.transform import  resize, rescale
from skimage.color import rgb2gray
import numpy as np
from os import listdir, path
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os,os.path
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, BatchNormalization
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import MobileNetV2

preprocessing

In [19]:
SEED = 42
IMG_SIZE = (224, 224)  # ResNet50 input size
BATCH_SIZE = 32
EPOCHS = 15
LEARNING_RATE = 0.001

np.random.seed(SEED)
tf.random.set_seed(SEED)
data='/kaggle/input/plantvillage-dataset/color'

def create_dataframe(data_path):
    filepaths = []
    labels = []
    folds = os.listdir(data_path)

    for fold in folds:
        f_path = os.path.join(data_path, fold)
        imgs = os.listdir(f_path)

        for img in imgs:
            img_path = os.path.join(f_path, img)

            filepaths.append(img_path)
            labels.append(fold)

    fseries = pd.Series(filepaths, name='Filepaths')
    lseries = pd.Series(labels, name='Labels')

    return pd.concat([fseries, lseries], axis=1)                                         
df = create_dataframe(data)

In [34]:
train_df, dummy_df = train_test_split(df,  train_size= 0.8, shuffle= True, random_state= 42)
valid_df, test_df = train_test_split(dummy_df,  train_size= 0.5, shuffle= True, random_state= 42)
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=25,
    width_shift_range=0.25,
    height_shift_range=0.25,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

valid_datagen = ImageDataGenerator(rescale=1./255) 
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepaths',
    y_col='Labels',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=True,
    seed=42
)
valid_generator = valid_datagen.flow_from_dataframe(
    dataframe=valid_df,
    x_col='Filepaths',
    y_col='Labels',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)
test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    x_col='Filepaths',
    y_col='Labels',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)
label_encoder = LabelEncoder()
df['label_encoded'] = label_encoder.fit_transform(df['Labels'])

num_classes = len(label_encoder.classes_)
print(f"\nNumber of classes: {num_classes}")
print("\nChecking class distribution...")

class_counts = df['Labels'].value_counts()
print(f"Classes with < 3 samples: {sum(class_counts < 3)}")

valid_classes = class_counts[class_counts >= 3].index
df_filtered = df[df['Labels'].isin(valid_classes)].copy()

print(f"Original dataset: {len(df)} images, {len(df['Labels'].unique())} classes")
print(f"Filtered dataset: {len(df_filtered)} images, {len(df_filtered['Labels'].unique())} classes")

label_encoder = LabelEncoder()
df_filtered['label_encoded'] = label_encoder.fit_transform(df_filtered['Labels'])
num_classes = len(label_encoder.classes_)   # <-- FIXED

X = df_filtered['Filepaths'].values
y = df_filtered['label_encoded'].values


X_train_val, X_test, y_train_val, y_test = train_test_split(
    X, y, test_size=0.2, random_state=SEED, stratify=y
)


X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val, test_size=0.2, random_state=SEED, stratify=y_train_val
)

NUM_CLASSES = len(train_generator.class_indices)
print("classes:", train_generator.class_indices)

print(f"\nDataset split:")
print(f"Train: {len(X_train)} images")
print(f"Validation: {len(X_val)} images")
print(f"Test: {len(X_test)} images")

NameError: name 'DATA_DIR' is not defined

In [23]:
# تأكد إن عندك tensorflow 2.x
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models, optimizers, callbacks
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input


In [29]:
base_model = MobileNetV2(
    input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3),
    include_top=False,
    weights='/kaggle/input/weights/mobilenet_v2_weights_tf_dim_ordering_tf_kernels_1.0_224_no_top.h5'
)
base_model.trainable = False  # freeze initially

# بناء الرأس (head)
inputs = tf.keras.Input(shape=(IMG_SIZE[0], IMG_SIZE[1], 3))
x = base_model(inputs, training=False)  # important: training=False عند استخدام layers مثل BatchNorm
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.3)(x)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(NUM_CLASSES, activation='softmax')(x)

model = tf.keras.Model(inputs, outputs)
model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_6 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 256)            │       327,936 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 38)             │         9,766 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,595,686 (9.90 MB)

 Trainable params: 337,702 (1.29 MB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [31]:
initial_lr = 1e-3
model.compile(
    optimizer=optimizers.Adam(learning_rate=initial_lr),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

ckpt_path = "mobilenetv2_best.h5"
cb_list = [
    callbacks.ModelCheckpoint(ckpt_path, monitor='val_accuracy', save_best_only=True, verbose=1),
    callbacks.EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True, verbose=1),
    callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1)
]


In [35]:
EPOCHS_HEAD = 8  # عدد epochs مبدئي للـ head
history_head = model.fit(
    train_generator,
    validation_data=valid_generator,
    epochs=EPOCHS_HEAD,
    callbacks=cb_list
)


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/8
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.6512 - loss: 1.2633
Epoch 1: val_accuracy improved from -inf to 0.90479, saving model to mobilenetv2_best.h5


1358/1358 ━━━━━━━━━━━━━━━━━━━━ 1572s 1s/step - accuracy: 0.6513 - loss: 1.2630 - val_accuracy: 0.9048 - val_loss: 0.3148 - learning_rate: 0.0010
Epoch 2/8
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 0s 970ms/step - accuracy: 0.8336 - loss: 0.5194
Epoch 2: val_accuracy improved from 0.90479 to 0.91731, saving model to mobilenetv2_best.h5


1358/1358 ━━━━━━━━━━━━━━━━━━━━ 1439s 1s/step - accuracy: 0.8336 - loss: 0.5194 - val_accuracy: 0.9173 - val_loss: 0.2605 - learning_rate: 0.0010
Epoch 3/8
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 0s 961ms/step - accuracy: 0.8521 - loss: 0.4565
Epoch 3: val_accuracy improved from 0.91731 to 0.92026, saving model to mobilenetv2_best.h5


1358/1358 ━━━━━━━━━━━━━━━━━━━━ 1426s 1s/step - accuracy: 0.8521 - loss: 0.4565 - val_accuracy: 0.9203 - val_loss: 0.2399 - learning_rate: 0.0010
Epoch 4/8
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 0s 932ms/step - accuracy: 0.8576 - loss: 0.4295
Epoch 4: val_accuracy improved from 0.92026 to 0.92431, saving model to mobilenetv2_best.h5


1358/1358 ━━━━━━━━━━━━━━━━━━━━ 1388s 1s/step - accuracy: 0.8576 - loss: 0.4295 - val_accuracy: 0.9243 - val_loss: 0.2185 - learning_rate: 0.0010
Epoch 5/8
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 0s 971ms/step - accuracy: 0.8633 - loss: 0.4224
Epoch 5: val_accuracy improved from 0.92431 to 0.93241, saving model to mobilenetv2_best.h5


1358/1358 ━━━━━━━━━━━━━━━━━━━━ 1461s 1s/step - accuracy: 0.8633 - loss: 0.4224 - val_accuracy: 0.9324 - val_loss: 0.2065 - learning_rate: 0.0010
Epoch 6/8
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 0s 972ms/step - accuracy: 0.8695 - loss: 0.4025
Epoch 6: val_accuracy did not improve from 0.93241
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 1439s 1s/step - accuracy: 0.8695 - loss: 0.4025 - val_accuracy: 0.9184 - val_loss: 0.2455 - learning_rate: 0.0010
Epoch 7/8
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 0s 976ms/step - accuracy: 0.8705 - loss: 0.4023
Epoch 7: val_accuracy did not improve from 0.93241
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 1444s 1s/step - accuracy: 0.8705 - loss: 0.4023 - val_accuracy: 0.9258 - val_loss: 0.2144 - learning_rate: 0.0010
Epoch 8/8
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 0s 978ms/step - accuracy: 0.8721 - loss: 0.3963
Epoch 8: val_accuracy did not improve from 0.93241

Epoch 8: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 1447s 1s/step - accuracy: 0.8721 - lo

In [37]:
model.save("mobilenetv2.keras")